# Import and Settings

In [36]:
import numpy as np
import scipy

## Directed Graph

In [55]:
directed_graph = [[0,1,1,1],[0,0,1,1],[1,0,0,0],[1,0,1,0]]
directed_graph

[[0, 1, 1, 1], [0, 0, 1, 1], [1, 0, 0, 0], [1, 0, 1, 0]]

# Processing

## Converting Graph

In [56]:
# Assumes it is a square matrix
def convert_graph_to_link_matrix(graph):
    transformed_graph = []
    for row in graph:
        col = []
        sum_row = sum(row)
        for val in row:
            col.append(val / sum_row)
        transformed_graph.append(col)
    transformed_graph = np.array(transformed_graph)
    transformed_graph = transformed_graph.transpose()
    return transformed_graph

In [57]:
A = convert_graph_to_link_matrix(directed_graph)
A

array([[0.        , 0.        , 1.        , 0.5       ],
       [0.33333333, 0.        , 0.        , 0.        ],
       [0.33333333, 0.5       , 0.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

## Eigen Values and Vector

In [46]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A)

In [58]:
for val in eigenvalues:
    print(val)

(1.000000000000002+0j)
(-0.36062333350611053+0.4109755454950538j)
(-0.36062333350611053-0.4109755454950538j)
(-0.278753332987779+0j)
